In [1]:
# 구글 드라이브 colab과 연동
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## CNN을 활용한 감정분석 

- tf.estimator()활용 

In [0]:
import sys
import os
import numpy as np
import json

import tensorflow as tf
from tensorflow import keras

In [5]:
# 이전장에서 학습에 필요한 디렉터리 설정 및 학습/평가 데이터를 불러온다. 
DATA_IN_PATH = "/content/gdrive/My Drive/study/kaggle/"
DATA_OUT_PATH = "/content/gdrive/My Drive/study/kaggle/"
INPUT_TRAIN_DATA_FILE_NAME = "word2vec-nlp-tutorialtrain_input.npy"
LABEL_TRAIN_DATA_FILE_NAME = "word2vec-nlp-tutorialtrain_label.npy"
INPUT_TEST_DATA_FILE_NAME = "word2vec-nlp-tutorialtest_input.npy"

DATA_CONFIGS_FILE_NAME = "word2vec-nlp-tutorialdata_configs.json"

train_input_data = np.load(open(DATA_IN_PATH+INPUT_TRAIN_DATA_FILE_NAME, 'rb'))
train_label_data = np.load(open(DATA_IN_PATH+LABEL_TRAIN_DATA_FILE_NAME, 'rb'))
test_input_data = np.load(open(DATA_IN_PATH+INPUT_TEST_DATA_FILE_NAME, 'rb'))

with open(DATA_IN_PATH+DATA_CONFIGS_FILE_NAME, 'r') as f:
  prepro_configs = json.load(f)
  print(prepro_configs.keys())

dict_keys(['vocab', 'vocab_size'])


In [0]:
# 파라미터 변수
RNG_SEED = 1234
BATCH_SIZE = 128
NUM_EPOCHS = 10000
VOCAB_SIZE = prepro_configs['vocab_size']
EMB_SIZE = 128
VALID_SPLIT = 0.2

from sklearn.model_selection import train_test_split
# 학습 데이터와 검증 데이터를 train_test_split()이용해서 나누기 
train_input, eval_input, train_label, eval_label = train_test_split(train_input_data, train_label_data, test_size =VALID_SPLIT, random_state = RNG_SEED )

# 전처리 학습을 위해 tf.data설정
def mapping_fn (X, Y=None):
  input, label = {'x':X}, Y
  return input, label

def train_input_fn():
  dataset = tf.data.Dataset.from_tensor_slices((train_input, train_label))
  dataset = dataset.shuffle(buffer_size = len(train_input))
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.map(mapping_fn)
  dataset = dataset.repeat(count = NUM_EPOCHS)

  iterator = dataset.make_one_shot_iterator()

  return iterator.get_next()

def eval_input_fn():
  dataset = tf.data.Dataset.from_tensor_slices((input_eval, label_eval))
  dataset = dataset.shuffle(buffer_size = len(input_eval))
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.map(mapping_fn)
  
  iterator = dataset.make_one_shot_iterator()

  return iterator.get_next()

### 합성곱 연산의경우 케라스 모듈 Conv1D를 활용
### (합성곱+맥스풀링)x3

총 세개의 합성곱을 사용/ 각각 필터를 다르게 해서 적용 /즉, kernel_size를 3,4,5로 다르게 해서 적용 / 그리고 출력값을 하나로 합친다. 

추가로 합성곱 신경망 이후에 맥스 풀링 층을 적용한다. 

In [0]:
def model_fn(features, labels, mode):

  TRAIN = mode = tf.estimator.ModeKeys.TRAIN
  EVAL = mode = tf.estimator.ModeKeys.EVAL
  PREDICT = mode = tf.estimator.ModeKeys.PREDICT

  # embedding layer선언
  embedding_layer = keras.layers.Embedding(
                                            VOCAB_SIZE,
                                           EMB_SIZE)(features['x'])
                                    
  # embedding layer에 대한 output에 대해서 dropout을 선언
  dropout_emb = keras.layers.Dropout(rate=0.5)(embedding_layer)

  # filters = 128 , kernel_size = 3,4,5
  # 길이가 3,4,5인 128개의 다른 필터를 생성, 3,4,5 gram의 효과처럼 다양한 각도에서 문장을 보는 효과
  #conv1d는 (배치크기, 길이, 채널)로 입력값을 받는데, 배치사이즈: 문장숫자 | 길이: 각 문장의 단어 개수| 채널 : 임베딩 출력 차원수

  conv1 = keras.layers.Conv1D(
                              filters = 128,
                              kernel_size = 3,
                              padding = 'valid',
                              activation = tf.nn.relu)(dropout_emb)

  pool1 = keras.layers.GlobalAveragePooling1D()(conv1)

  conv2 = keras.layers.Conv1D(
                              filters = 128,
                              kernel_size = 4,
                              padding = 'valid',
                              activation = tf.nn.relu)(dropout_emb)

  pool2 = keras.layers.GlobalAveragePooling1D()(conv2)

  conv3 = keras.layers.Conv1D(
                              filters = 128,
                              kernel_size = 5,
                              padding = 'valid',
                              activation = tf.nn.relu)(dropout_emb)
  
  pool3 = keras.layers.GlobalAveragePooling1D()(conv3)

  concat = keras.layers.concatenate([pool1, pool2, pool3])

  hidden = keras.layers.Dense(250, activation=tf.nn.relu)(concat)
  dropout_hidden = keras.layers.Dropout(rate=0.5)(hidden)
  logits = keras.layers.Dense(1, name = 'logit')(dropout_hidden)
  logits = tf.squeeze(logits, axis = -1)

  #최종적으로 학습, 검증, 평가의 단계로 나누어 활용
  if PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions = {'prob':tf.nn.sigmoid(logits)})

  loss = tf.losses.sigmoid_cross_entropy(labels, logits)

  if EVAL:
    pred = tf.nn.sigmoid(logits)
    accuracy = tf.metrics.accuracy(lables, tf.round(pred))
    return tf.estimator.EstimatorSpec(mode = mode, loss = loss, eval_metric_ops = {'acc':accuracy})

  if TRAIN:
    global_step = tf.train.get_global_step()
    train_op = tf.train.AdamOptimizer(0.001).minimize(loss, global_step)

    return tf.estimator.EstimatorSpec(mode=mode, train_op = train_op, loss = loss)

In [23]:
#모델 검증
model_dir = os.path.join(os.getcwd(), "/content/gdrive/My Drive/study/kaggle/data_out/checkpoint/cnn/")
os.makedirs(model_dir, exist_ok=True)

graph = tf.Graph()
with graph.as_default():
    v = tf.get_variable("model", shape=(100, 100), dtype=tf.float32)

save = tf.train.CheckpointSaverHook("model_dir", 10)
with graph.as_default():
    tf.train.create_global_step()
    a = tf.constant(1)
    with tf.train.MonitoredSession(hooks=[save]) as sess:
        sess.run(a)
# Estimator객체 생성
cnn_est = tf.estimator.Estimator(model_fn, model_dir=model_dir)
cnn_est.train(train_input_fn)
cnn_est.evaluate(eval_input_fn)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model_dir/model.ckpt.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/content/gdrive/My Drive/study/kaggle/data_out/checkpoint/cnn/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.serve

ValueError: ignored

In [19]:
pip list

Package                  Version              
------------------------ ---------------------
absl-py                  0.8.0                
alabaster                0.7.12               
albumentations           0.1.12               
altair                   3.2.0                
astor                    0.8.0                
astropy                  3.0.5                
atari-py                 0.1.15               
atomicwrites             1.3.0                
attrs                    19.2.0               
audioread                2.1.8                
autograd                 1.3                  
Babel                    2.7.0                
backcall                 0.1.0                
backports.tempfile       1.0                  
backports.weakref        1.0.post1            
beautifulsoup4           4.6.3                
bleach                   3.1.0                
blis                     0.2.4                
bokeh                    1.0.4                
boto         